# Environment Preparation

In [1]:
# !pip install ../input/dlib-whl/dlib-19.19.0-cp36-cp36m-linux_x86_64.whl
# !dpkg -i  /kaggle/input/ffmpeg-debs/ffmpeg/*.deb
!pip install /kaggle/input/mtcnn-whl/facenet_pytorch-2.0.1-py3-none-any.whl

Processing /kaggle/input/mtcnn-whl/facenet_pytorch-2.0.1-py3-none-any.whl


In [2]:
import cv2
import os
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from facenet_pytorch import MTCNN
import traceback

pd.options.display.max_columns = 500
pd.options.display.max_rows = 2000

In [3]:
detector = MTCNN(select_largest=False, margin=50, keep_all=True, post_process=False)

# Variable Declaration

In [4]:
test_video_path = "/kaggle/input/deepfake-detection-challenge/test_videos/"
train_video_path = "/kaggle/input/deepfake-detection-challenge/train_sample_videos/"

# Utils Function

In [5]:
def extract_face_from_video(path_to_video, detector, save_dir=".", faces_count=5, crop_size=(224, 224)):
    path = Path(path_to_video)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    v_cap = cv2.VideoCapture(path_to_video)
    i = 0
    faces_path = []
    while True:
        success, frame = v_cap.read()
        if not success:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        # Detect face
        faces = detector(frame)
        if not faces is None:
            for face in faces:
                face = face.permute(1, 2, 0).int().numpy().astype(np.uint8)
                face = Image.fromarray(face)
                face = face.resize(crop_size, Image.ANTIALIAS)
                save_path = f"{save_dir}/{path.stem}{i:03d}.jpg"
                face.save(save_path)
                faces_path.append(save_path)
                i += 1
        if i >= faces_count:
            break
    return faces_path

In [6]:
# from https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/

def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape)

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (x.max()-x.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]
    radial_prof = tbin / nr

    return radial_prof

In [7]:
def get_psd1D(path_to_img, epsilon = 1e-8):
    img = cv2.imread(path_to_img,0)
    # Calculate FFT
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    fshift += epsilon
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = azimuthalAverage(magnitude_spectrum)
    return psd1D

# Predict

In [8]:
extract_face_from_video("/kaggle/input/deepfake-detection-challenge/test_videos/ahjnxtiamx.mp4", detector=detector)

['./ahjnxtiamx000.jpg',
 './ahjnxtiamx001.jpg',
 './ahjnxtiamx002.jpg',
 './ahjnxtiamx003.jpg',
 './ahjnxtiamx004.jpg']

In [9]:
#包导入
%matplotlib inline
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC

In [10]:
def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [11]:
model = load_object("../input/svm-model/svm.pkl")

In [ ]:
df = pd.DataFrame(columns=['filename', 'label'])
count = 1
for i, video in tqdm(enumerate(os.listdir(test_video_path))):
    print(i, video)
    imgs = extract_face_from_video(f"{test_video_path}/{video}", detector=detector)
#     print(imgs)
    if len(imgs)<1:
        proba = 1
    else:
        probas = []
        for img in imgs:
            try:
                psd1D = get_psd1D(img)
                psd1D = psd1D[:156].reshape(-1, 156)
                probas.append(model.predict_proba(psd1D)[0][1])
            except:
                pass
        proba = np.mean(probas)
    print(proba)
    df.loc[i] = [video, proba]
df

0it [00:00, ?it/s]

0 ywauoonmlr.mp4


1it [00:02,  2.32s/it]

0.794101620665812
1 sfsayjgzrh.mp4


2it [00:05,  2.68s/it]

...

0.7514610534696301
109 ryxaqpfubf.mp4


110it [05:43,  2.97s/it]

0.7761070821431384
110 yllztsrwjw.mp4


111it [05:47,  3.03s/it]

0.760924144324908
111 gfcycflhbo.mp4


In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
df.to_csv("submission.csv", index=False)